# Building an Anime Recommender System

In [1]:
from animerec import datasets, utils
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Load Dataset and Model

In [2]:
import sys
sys.modules['datasets'] = datasets
sys.modules['utils'] = utils

# Load dataset for cindex, id_to_anime, etc.
pickle_in = open("data/MyAnimeList.pickle", 'rb')
data = pickle.load(pickle_in)
pickle_in.close()

# Load model trained on server.
pickle_in = open("animerec/model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

### 3. Print Features in Model

In [3]:
model.print_features(data)

Feature 1:
	max : (+4.0) Suzumiya Haruhi no Yuuutsu
	min : (-6.0) Naruto: Shippuuden
Feature 2:
	max : (+5.0) .hack//Sign
	min : (-3.1) Tsubasa Chronicle 2nd Season
Feature 3:
	max : (+4.7) Sword Art Online
	min : (-3.3) Higashi no Eden Movie II: Paradise Lost
Feature 4:
	max : (+7.0) Pingu in the City
	min : (-4.1) Lucky☆Star
Feature 5:
	max : (+4.5) Bobobo-bo Bo-bobo
	min : (-4.4) Aki-Sora
Feature 6:
	max : (+5.9) Ore no Imouto ga Konnani Kawaii Wake ga Nai. Specials
	min : (-5.2) School Days
Feature 7:
	max : (+3.6) Boku no Pico
	min : (-7.6) School Days
Feature 8:
	max : (+3.6) Vampire Knight: Guilty
	min : (-3.9) Hachimitsu to Clover
Feature 9:
	max : (+5.4) Dragon Ball GT
	min : (-4.1) Aku no Hana
Feature 10:
	max : (+6.9) One Piece
	min : (-2.9) Princess Princess
Feature 11:
	max : (+4.2) School Days
	min : (-5.7) Sword Art Online II
Feature 12:
	max : (+4.6) Neon Genesis Evangelion
	min : (-4.2) Shinsekai yori
Feature 13:
	max : (+4.3) Bokusatsu Tenshi Dokuro-chan
	min : (-3.7)

### 4. Webscrape current profile

In [4]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, data.cindex)

Scraped 300 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=0&status=7
Scraped 148 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=300&status=7


### 5. Create prediction

In [5]:
xhat = model.predict(x)

In [6]:
prediction = utils.prediction_to_dataframe(xhat, myprofile, data.cindex)

print('Top 10 Anime Predictions')
for i in prediction.sort_values(ascending=False).head(10).index:
    str_ = (
        f"\t({prediction[i]:2.4f}) {data.get_anime_by_id(i).title}"
    )
    print(str_)
    
print('\nBottom 10 Anime Predictions')
for i in prediction.sort_values(ascending=False).tail(10).index:
    str_ = (
        f"\t({prediction[i]:2.4f}) {data.get_anime_by_id(i).title}"
    )
    print(str_)

Top 10 Anime Predictions
	(11.1159) Nana
	(10.8133) Kenpuu Denki Berserk
	(9.5002) Paradise Kiss
	(9.2069) Ginga Eiyuu Densetsu
	(9.1089) Devilman: Crybaby
	(8.9505) Nodame Cantabile Finale
	(8.8477) Koe no Katachi
	(8.7421) Yoru wa Mijikashi Arukeyo Otome
	(8.7103) Nodame Cantabile: Paris-hen
	(8.6801) Chihayafuru

Bottom 10 Anime Predictions
	(0.8278) Masou Gakuen HxH
	(0.8004) DRAMAtical Murder
	(0.7420) Suki na Mono wa Suki Dakara Shou ga Nai!!
	(0.6876) Angel&#039;s Feather
	(0.5967) Makura no Danshi
	(0.4612) 07-Ghost
	(0.0493) Zombie-Loan
	(-0.0565) Junjou Romantica 2
	(-0.6943) Gakuen Heaven
	(-0.7745) Junjou Romantica
